In [ ]:
def enregistrement_npy(dt=256):
  for machine in df.Machine.unique():
      #Génération de spectres
      df_train = df[ (df['Machine']== machine) & (df['train_test'] == 'train') ] # sélection de la machine 
      df_valid = df[ (df['Machine']== machine) & (df['train_test'] == 'test') ] # sélection de la machine 

      #partie non testée pour adapter l'URL au google drive
      URLS_train=[]
      URLS_valid=[]
      for lignes in df_train['URL']:
        URLS_train.append(str("/content/drive/MyDrive/Datascience projet son/"+lignes))
      for lignes in df_valid['URL']:
        URLS_valid.append(str("/content/drive/MyDrive/Datascience projet son/"+lignes))

      data_train = load_data(URLS_train, dt=dt, T_max=10,normalize=True)
      data_valid = load_data(URLS_valid, dt=dt, T_max=10,normalize=True)
      
      # on cherche la puissance de 2 la plus proche du nb fréquence
      carre=2
      while carre<data_train.shape[2]:
          carre*=2
      
      freq=int(carre/2)
      
      #Reduction de dimension
      sel=downsizing()      
      sel.fit(data_train,seuil_db=False,nb_freq=freq)
      data_train=sel.transform(data_train,normalize=False)
      data_valid=sel.transform(data_valid,normalize=False)

      #Enregistrement
      nom_fichier_train=str(str(machine)+"_train_norm_"+str(dt)+"_"+str(freq)+".npy")
      nom_fichier_test=str(str(machine)+"_test_norm_"+str(dt)+"_"+str(freq)+".npy")

      np.save(nom_fichier_train,data_train.flatten())
      np.save(nom_fichier_test,data_valid.flatten())
  return


In [ ]:
def load_data(audio_path, dt=256, T_max=10, normalize=True):
    """
    Permet de charger un spectre logmel depuis un path. 
    
    params: 
    
    dt: float /int. si [0:1[ alors dt est considéré comme un pas de découpage en seconde 
        si >=1 alors doit être un int réprésentant le nombre de découpages souhaité
    
    """
    X_audio=[]
    
    if dt<1: # on veut selectionner le pas en seconde
        with tqdm(total=len(audio_path)) as pbar: # code pour afficher une barre de chargement
            for i, a_path in enumerate(audio_path) :
                pbar.update(1) # permet de mettre la barre de chargement à jour en ajoutant 1 itération
                # Load the audio file
                data, fe = librosa.load(a_path, sr=None)

                # For the audio > T_max : Use just the fist T_max seconde, to have the right shape. 
                if len(data)>= T_max*fe:
                    data = data[:int(T_max*fe)]

                # For the audio < T_max : Add in the signal a zeros vector, to have the right shape.
                else :
                    data = np.concatenate([data, np.zeros(int(T_max*fe - len(data)))])

                # Apply the logMelSpectrogram function.    
                spectre_audio = logMelSpectrogram(data, fe, dt)
                if normalize : 
                    spectre_audio=(np.array(spectre_audio)-np.array(spectre_audio).min())/(np.array(spectre_audio).max()-np.array(spectre_audio).min())
                    X_audio.append(spectre_audio)
            
        
    else: # on veut selectionner un nombre de dt 
        with tqdm(total=len(audio_path)) as pbar:
            for a_path in audio_path :
                pbar.update(1)
                # Load the audio file
                data, fe = librosa.load(a_path, sr=None)

                # For the audio > T_max : Use just the fist T_max seconde, to have the right shape. 
                if len(data)>= T_max*fe:
                    data = data[:int(T_max*fe)]

                # For the audio < T_max : Add in the signal a zeros vector, to have the right shape.
                else :
                    data = np.concatenate([data, np.zeros(int(T_max*fe - len(data)))])

                # Apply the logMelSpectrogram function. 

                spectre_audio = logMelSpectrogram(data[:(len(data)//dt)*dt], fe, np.around(T_max/dt,3))
                spectre_audio=spectre_audio[:dt,:] # on tronque quand même au cas où nous avons dépasser le nombre de dt. 
                if normalize : 
                    spectre_audio=(np.array(spectre_audio)-np.array(spectre_audio).min())/(np.array(spectre_audio).max()-np.array(spectre_audio).min())

                X_audio.append(spectre_audio)
                if np.array(X_audio).shape[1]!=dt:
                  raise Exception("Erreur le nombre de dt réalisé ne coincide pas avec la demande, changer le nombre demandé \n" "format actuel de sortie",np.array(X_audio).shape)
    
        return np.array(X_audio)


In [ ]:
class spectrum_generator():
    def __init__(self):
        self.level=0
        self.v_translate=0
        self.cut=None
        self.nb_freq=0
        self.nb_dt=0
        self.nb_samples_tot=0
        self.nb_samples_target=0
        
    def fit(self,X,y,level=0,v_translate=0,dt_cut=None):
        #récupération des paramètres
        self.level=level
        self.v_translate=v_translate
        self.nb_samples_tot,self.nb_dt,self.nb_freq=X.shape
        
        if dt_cut=='first':
            self.cut=0
        elif dt_cut=='last':
            self.cut=-1
        elif dt_cut=='both':  
            self.cut=[0,-1]
        else :
            self.cut=None
        
    def flow_from_numpy(self,X,y,batchsize=32,pourcent_modif=0.1):
        #initialisation de la selection aléatoire des samples
        selector=np.random.random(batchsize)*self.nb_samples_tot
        selector=selector.astype(int)
        transform_sel=np.random.random(int(pourcent_modif*batchsize))*selector.shape
        transform_sel=transform_sel.astype(int)
        #sélection des samples
        X_selected=np.array(X)[selector]
        y_selected=np.array(y)[selector]
        #transformation aléatoire des samples via une des fonctions
        for modif in transform_sel:
            choix_modif=int(np.random.random(1)*3)
            if choix_modif<=1:
                X_selected[modif,:,:]=X_selected[modif,:,:]*(1+(self.level*np.random.random(1))-(self.level*np.random.random(1))) # on multiplie le spectre par une valeur 1+- self.level
           
            elif (choix_modif>1) and (choix_modif<=2):
                X_selected[modif,:,:]=X_selected[modif,:,:]+(self.v_translate*np.random.random(1))-(self.v_translate*np.random.random(1))# on ajoute au spectre une valeur 1+- self.v_translate
            
            elif (choix_modif>2) and (choix_modif<=3):
                for dt in self.cut:
                    X_selected[modif,dt,:]=X_selected[modif,dt,:]*0
            else:
                X_selected[modif,:,:]=X_selected[modif,:,:]
        
        return X_selected,y_selected
        
    def add_samples(self,X,y,pourcent_add=0.1, labels='all'):
        self.nb_samples_tot=y.shape[0]
        if labels=='all':
            self.nb_samples_target=self.nb_samples_tot
        else:
            self.nb_samples_target=y[y==labels].shape[0]
        
        if self.nb_samples_target ==self.nb_samples_tot : 
            #initialisation de la selection aléatoire des samples
            selector=np.random.random(int(pourcent_add*self.nb_samples_tot))*self.nb_samples_tot
            selector=selector.astype(int)
            #sélection des samples
            X_selected=np.array(X)[selector]
            y_selected=np.array(y)[selector]
        
        else :
            selector=np.random.random(int(pourcent_add*self.nb_samples_target))*self.nb_samples_target
            selector=selector.astype(int)
            #sélection des samples
            X_selected=np.array(X)[y==labels][selector]
            y_selected=np.array(y)[y==labels][selector]
        
        #transformation aléatoire des samples via une des fonctions
        for modif in range(X_selected.shape[0]):
            choix_modif=int(np.random.random(1)*3)
            if choix_modif<=1:
                X_selected[modif,:,:]=X_selected[modif,:,:]*(1+self.level)
           
            elif (choix_modif>1) and (choix_modif<=2):
                X_selected[modif,:,:]=X_selected[modif,:,:]*(1-self.level)  
            
            elif (choix_modif>2) and (choix_modif<=3):
                for dt in self.cut:
                    X_selected[modif,dt,:]=X_selected[modif,dt,:]*0
            else:
                X_selected[modif,:,:]=X_selected[modif,:,:]
        
        return np.concatenate((X,X_selected),axis=0),np.concatenate((y,y_selected),axis=0)   
        


In [ ]:
class downsizing():
    def __init__(self):
        self.selector=0
        self.datamax=0
    
    def fit(self,X,seuil_db=False,nb_freq=False):
        self.datamax=X.max(axis=1).max(axis=0)
        if (seuil_db!=False) and (nb_freq!=False):
                return print('Renseignez un seuil_db OU un nombre de fréquences à garder mais pas les deux')

        elif seuil_db=='brute':
            #Méthode brute
            data_s=X.sum(axis=1).sum(axis=0)
            data_s.shape
            self.selector=np.where(data_s>0,True,False)  
        elif seuil_db!=False:  
            self.selector=np.where(self.datamax>seuil_db,True,False)

        elif nb_freq!=False:  
            self.selector=np.where(self.datamax>np.sort(self.datamax)[-(nb_freq+1)],True,False) # on est très restrictif dans un premier temps 

            if np.sum(self.selector)<nb_freq:
                self.selector=np.where(self.datamax>=np.sort(self.datamax)[-(nb_freq+1)],True,False) # si nous avons été trop restrictif, alors nous élargissons la selection puis nous bouclons pour supprimer le surplus

            iter=100
            while np.sum(self.selector)>nb_freq and iter>0: # En cas de multiples valeurs similaires on boucle pour les supprimer et ne garder que la dim voulue
                print('Trop de fréquences choisies, suppression de {:} fréquences pour atteindre la taille cible'.format( self.selector.sum()-nb_freq))
                self.selector[np.argsort(np.sort(self.datamax[self.selector]))[-1]]=False  # on récupère la position du plus petit élément de datamax pour la supprimer du sélector
                iter-=1 # on met 10 itérations pour éviter de boucler à l'infini en cas d'erreur
        print('nombre de fréquences gardées: ', self.selector.sum())
        
    def transform(self,X,normalize=False):
        
        if normalize:
            def normalisation(ligne2D):
                return (np.array(ligne2D)-np.array(ligne2D).min())/(np.array(ligne2D).max()-np.array(ligne2D).min())  
            
            X_select=X[:,:,self.selector]
            X_select=np.array([ normalisation(samples) for samples in X_select ])
            return X_select
        
        else :
            return X[:,:,self.selector]
    
    
    def get_selector(self,seuil_db):
        self.selector=np.where(self.datamax>seuil_db,True,False)
        return self.selector
    
    
    def get_datamax(self):
        return self.datamax


In [ ]:
def logMelSpectrogram(audio, fe, dt):
    stfts = np.abs(librosa.stft(audio,n_fft = int(dt*fe),hop_length = int(dt*fe),center = True)).T
    num_spectrogram_bins = stfts.shape[-1]
    # MEL filter
    linear_to_mel_weight_matrix = librosa.filters.mel(sr=fe,n_fft=int(dt*fe) + 1,n_mels=num_spectrogram_bins,).T
    # Apply the filter to the spectrogram
    mel_spectrograms = np.tensordot(stfts,linear_to_mel_weight_matrix,1)
    return np.log(mel_spectrograms + 1e-6)

In [ ]:
def plot_logMelSpectrogram(audio, fe, dt=0.025):
    sns.heatmap(np.rot90(logMelSpectrogram(audio, fe, dt)), cmap='inferno', vmin = -6)
    loc, labels = plt.xticks()
    l = np.round((loc-loc.min())*len(audio)/fe/loc.max(), 2)
    plt.xticks(loc, l)
    plt.yticks([])
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Mel)")

In [ ]:
def wave_path(machine,folder,type_audio,k):
    machine_path='./'+machine+'/'
    type_path=folder+'/'
    zeros='00000000'+str(k)
    file_name=type_audio+'_id_00_'+zeros[-8:]+'.wav'
    file_path=machine_path+type_path+file_name
    rate,data = wave.read(file_path)
    graph_name=file_name.split('.')[0]+'_spectre.jpg'
    save_path=machine_path+'Spectres/'+graph_name
    n = data.size
    duree = 1.0*n/rate
    return file_path,save_path,n,duree,rate,data

In [ ]:
def tracerSpectre_op(data,rate,debut,duree):
    start = int(debut*rate)
    stop = int((debut+duree)*rate)
    # Récupération de la valeur absolue de la transformée de fourier sur l'échantillon
    spectre = np.absolute(fft(data[start:stop]))
    # normalisation des valeurs 0/1
    spectre = spectre/spectre.max()
    # On crée les labels des abscices 
    n = spectre.size
    freq=np.linspace(1/n,1,n)*rate
    # On trace un barplot avec x=freq,y=spectre
    plt.vlines(freq,[0],spectre,'r')
    plt.xlabel('f (Hz)')
    plt.ylabel('A')
    plt.axis([0,0.5*rate,0,1])
    plt.grid()

In [ ]:
def Fourier(data,rate,debut,duree):
    start = int(debut*rate)
    stop = int((debut+duree)*rate)
    # Récupération de la valeur absolue de la transformée de fourier sur l'échantillon
    spectre = np.absolute(fft(data[start:stop]))
    # normalisation des valeurs 0/1
    spectre = spectre/spectre.max()
    # On crée les labels des abscices 
    n = spectre.size
    freq=np.linspace(1/n,1,n)*rate
    return freq,spectre

In [ ]:
def Frequences(data,rate):
    n = data.size
    duree = 1.0*n/rate
    start = int(0)
    stop = int(duree*rate)
    # Récupération de la valeur absolue de la transformée de fourier sur l'échantillon
    spectre = np.absolute(fft(data[start:stop]))
    # normalisation des valeurs 0/1
    spectre = spectre/spectre.max()
    return spectre

In [ ]:
def Dossier_data(DOSSIER):
    """
    Cette fonction permet de parcourir les fichiers contenu 
    dans le DOSSIER 'DOSSIER' QUE VOUS SOUHAITEZ du notebook d'en extraire
    les informations et de générer un PandaDataFrame
    """
    if DOSSIER!='':
        chemin=glob.glob(''+DOSSIER+'/**',recursive=True)
    else :
        chemin=glob.glob('/**',recursive=True)
    
    df=pd.DataFrame(chemin, columns=['URL'])
    df['URL2']=df['URL'].apply(lambda contenu: os.path.split(contenu)[0])
    df['fichier']=df['URL'].apply(lambda contenu: os.path.split(contenu)[1])
    df['train_test']=df['URL2'].apply(lambda contenu: os.path.split(contenu)[1])
    df['URL2']=df['URL2'].apply(lambda contenu: os.path.split(contenu)[0])
    df['Machine']=df['URL2'].apply(lambda contenu: os.path.split(contenu)[1])
    df=df[(df['Machine']!='')&(df['Machine']!='.')&(df['Machine']!='Datascience_projet_son')&(df['Machine']!='./archive')&(df['URL2']!='./archive')]
    df['Machine_ID']=df['fichier'].apply(lambda contenu: contenu.split('_')[2])
    df['son_type']=df['fichier'].apply(lambda contenu: contenu.split('_')[0])
    df=df.drop('URL2',axis=1)
    duree=[]
    nb_boucles=df.shape[0]
    tps1 = time.time()
    for i,URL in enumerate(df['URL']):
      tps2 = time.time()
      print('boucle: ',i+1 ,' / ', nb_boucles+1)
      print('temps écoulé:{:.2f} sec'.format(tps2 - tps1))
      print('temps restant:{:.0f} min'.format((tps2 - tps1)*(nb_boucles-i)/((i+1)*60)))
      duree.append(duration(URL))
      clear_output()
      if i%1000==0 : # on supprime le cache de la RAM toutes les 1000 itérations
        librosa.cache.clear()

    df['duree_son']=duree
    tps3 = time.time()
    print('temps total:',tps3 - tps1)
    return df

In [ ]:
def plot_loss(model_history,title=""):
    plt.figure(figsize=(12,10))
    plt.plot(model_history.history['loss'][1:],'orange' ,label = 'Train Loss',)
    plt.plot(model_history.history['val_loss'][1:],'g', label = 'Test Loss')
    plt.title(title + " , "+ machine+" " +str(dt) +" " + str(freq))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend()

In [ ]:
def plot_mse_with_predict(model_load):
    pred_3d=model_load.predict(data_valid)
    pred_3d=pred_3d.reshape(-1, dt, freq)

    #on calcule la MSE
    from sklearn.metrics import mean_squared_error
    from scipy import stats
    score_3D=[]
    with tqdm(total=y_norm_valid.shape[0]) as pbar:
        for pred, true in zip(pred_3d,data_valid):
            score_3D.append(mean_squared_error(true,pred))
            pbar.update(1)

    score_3D_graph=pd.DataFrame(y_norm_valid)
    score_3D_graph['MSE']=np.array(score_3D)
    score_3D_graph.columns=['son_type','MSE']
    print(stats.ttest_ind(score_3D_graph[score_3D_graph.son_type=='anomaly']['MSE'],score_3D_graph[score_3D_graph.son_type=='normal']['MSE']))

    score_3D_graph[score_3D_graph.son_type=='normal'].plot.hist(alpha=0.5)
    plt.title('normal')
    score_3D_graph[score_3D_graph.son_type=='anomaly'].plot.hist(alpha=0.5)
    plt.title('anomaly');

In [ ]:
def rebuild_heatmap():
    nb_to_plot=6
    select=[np.random.random(nb_to_plot)*y_norm_valid.shape[0]]
    select=np.array(select).astype('int').reshape(nb_to_plot)
    plt.figure(figsize=(20,4))
    for i in range (1,nb_to_plot+1):
        ax1=plt.subplot(2,nb_to_plot,i)
        # ax1.imshow(data_valid_cut[select[i-1]])
        sns.heatmap(data_valid[select[i-1]])
        plt.title(y_norm_valid[select[i-1]] + ' original ')
        ax1.get_yaxis().set_visible(False)
        ax1.get_xaxis().set_visible(False)
        ax2=plt.subplot(2,nb_to_plot,i+nb_to_plot)
        # ax2.imshow(pred_3d[select[i-1]])
        sns.heatmap(pred_3d[select[i-1]])
        plt.title(y_norm_valid[select[i-1]] + ' reconstitution ')
        ax2.get_yaxis().set_visible(False)
        ax2.get_xaxis().set_visible(False)
        # plt.legend()
    plt.show()

In [ ]:
def visualisation(model_history,model_load=None,title="",noHeatmap=True):

    if model_history is not None:
        plot_loss(model_history,title)
    if noHeatmap:
        rebuild_heatmap()
    if model_load is not None: 
        plot_mse_with_predict(model_load)
    
    

